In [1]:
import torch

In [2]:
import re
from grid.clients.torch import TorchClient

/Users/jasonmancuso/anaconda/envs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-macosx-10.7-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
client = TorchClient(verbose=False)


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4
all parts .... ['', 'Users', 'jasonmancuso', '.openmined']
full path /
full path /Users/
full path /Users/jasonmancuso/
full path /Users/jasonmancuso/.openmined/

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/QmXkWUybbTnfvFH8SUcrug6RGTLYTB23gSockKLxueR1vQ...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmXVoAQN2hHf5VF5SfCV4kQqZqNuZi3PE1so3pMYFguSUS...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...SUCCESS!!!


UPDATE: Searching for IPFS nodes - 9 found overall - 4 are OpenMined workers          

SUCCESS: Found 4 OpenMined nodes!!!

Overloading Torch module
Overloading FloatTensor
Overloading DoubleTensor
Overloading HalfTensor
Overloading ByteTensor
Overloading CharTensor
Overloading ShortTensor
Overloading IntTensor
Overloading LongTensor
Overloading Variable
Overloading complete.


In [4]:
compute_nodes = [x for x in client if re.match('compute:', x)]
for x in compute_nodes:
    print(x)

compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4
compute:QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc


In [5]:
laptop = compute_nodes[0]
desktop = compute_nodes[1]

In [6]:
# Convenient way to send specific tensors to certain nodes
assign_tensors_to_nodes = {}
assign_tensors_to_nodes[desktop] = []
assign_tensors_to_nodes[laptop] = []

In [7]:
# Loop over available nodes
for node in compute_nodes:
    for i in range(5):
        # Just getting five randomly generated 2x2 matrices here
        assign_tensors_to_nodes[node].append(torch.FloatTensor(2,2))
        
# Fill desktop tensors with 1s
for tensor in assign_tensors_to_nodes[desktop]:
    tensor.fill_(1)
    
# Fill desktop tensors with 2s
for tensor in assign_tensors_to_nodes[laptop]:
    tensor.fill_(2)

In [8]:
# List of tensors -- tensors are local
# Sanity check -- we want desktop to have 2x2 matrices filled with 1s
assign_tensors_to_nodes[desktop]

[
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2], 
  1  1
  1  1
 [torch.FloatTensor of size 2x2]]

In [9]:
# List of tensors -- tensors are local
# Sanity check -- we want laptop to have 2x2 matrices filled with 2s
assign_tensors_to_nodes[laptop]

[
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2], 
  2  2
  2  2
 [torch.FloatTensor of size 2x2]]

In [10]:
# Multiply all tensors we want to send to desktop.
# Computation is done on this machine
accumulate = assign_tensors_to_nodes[desktop][0]
for x in assign_tensors_to_nodes[desktop][1:]:
    accumulate = accumulate.matmul(x)
print(accumulate)


 16  16
 16  16
[torch.FloatTensor of size 2x2]



In [11]:
# Multiply all tensors we want to send to laptop.
# Computation is done on this machine
accumulate = assign_tensors_to_nodes[laptop][0]
for x in assign_tensors_to_nodes[laptop][1:]:
    accumulate = accumulate.matmul(x)
print(accumulate)


 512  512
 512  512
[torch.FloatTensor of size 2x2]



In [12]:
# Send tensors to desktop node
for send_this_to_desktop in assign_tensors_to_nodes[desktop]:
    send_this_to_desktop.send_(desktop)

# Send tensors to laptop node
for send_this_to_laptop in assign_tensors_to_nodes[laptop]:
    send_this_to_laptop.send_(laptop)

In [13]:
# List of desktop tensors, now on desktop node
assign_tensors_to_nodes[desktop]

[[torch.FloatTensor - Locations:['compute:QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc']],
 [torch.FloatTensor - Locations:['compute:QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc']],
 [torch.FloatTensor - Locations:['compute:QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc']],
 [torch.FloatTensor - Locations:['compute:QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc']],
 [torch.FloatTensor - Locations:['compute:QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc']]]

In [14]:
# List of laptop tensors, now on laptop node
assign_tensors_to_nodes[laptop]

[[torch.FloatTensor - Locations:['compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']],
 [torch.FloatTensor - Locations:['compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']],
 [torch.FloatTensor - Locations:['compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']],
 [torch.FloatTensor - Locations:['compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']],
 [torch.FloatTensor - Locations:['compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4']]]

In [15]:
# Multiply all tensors on desktop.
# Computation is done on desktop.
accumulate_desk = assign_tensors_to_nodes[desktop][0]
for x in assign_tensors_to_nodes[desktop][1:]:
    accumulate_desk = accumulate_desk.matmul(x)
print(accumulate_desk)

[torch.FloatTensor with no dimension]



In [16]:
# Multiply all tensors on laptop.
# Computation is done on laptop.
accumulate_lap = assign_tensors_to_nodes[laptop][0]
for x in assign_tensors_to_nodes[laptop][1:]:
    accumulate_lap = accumulate_lap.matmul(x)
print(accumulate_lap)

[torch.FloatTensor with no dimension]



In [17]:
# Returns the data resulting from desktop computation
accumulate_desk.get_()


 16  16
 16  16
[torch.FloatTensor of size 2x2]

In [18]:
# Returns the data resulting from laptop computation
accumulate_lap.get_()


 512  512
 512  512
[torch.FloatTensor of size 2x2]

In [19]:
(2 * accumulate_desk ** 2 == accumulate_lap).all()

True

In [20]:
(accumulate_lap == accumulate).all()

True